In [5]:
import pandas as pd
import numpy as np

def get_dataframe_setdiff2d(df_new: pd.DataFrame, 
                            df_old: pd.DataFrame, 
                            rtol=1e-03, atol=1e-05) -> pd.DataFrame:
    """Returns set difference of two pandas DataFrames"""

    union_index = np.union1d(df_new.index, df_old.index)
    union_columns = np.union1d(df_new.columns, df_old.columns)

    new = df_new.reindex(index=union_index, columns=union_columns)
    old = df_old.reindex(index=union_index, columns=union_columns)

    mask_diff = ~np.isclose(new, old, rtol, atol)

    df_bool = pd.DataFrame(mask_diff, union_index, union_columns)

    df_diff = pd.concat([new[df_bool].stack(),
                         old[df_bool].stack()], axis=1)

    df_diff.columns = ["New", "Old"]

    return df_diff

"""
type_list = np.array([['Loan Type','loan_type'],['Category','category']]) 

df = pd.read_excel('source/training_data.xlsx')

for row in type_list:
    print(row[0])

df2 = df.groupby(['GENDER','District']).size().groupby(level=0).max() 
print(df2)

xdf = df['Grade'].unique().tolist()
print(np.mean(xdf,axis=0))

# given
df1 = pd.DataFrame({'A':[2,1,2],'C':[2,1,2]})
df2 = pd.DataFrame({'A':[1,1],'B':[1,1]})

diff = get_dataframe_setdiff2d(df1, df2)
print("diff:\n", diff, "\n")
"""

lowercase = lambda x: str(x).lower()
pay_df = pd.read_excel('source/pl_paymenet_date_snapshot_.xlsx',sheet_name = 'Sheet1')    
pay_df.rename(lowercase, axis='columns', inplace=True)

pay_df['snapshot_createdate'] = pd.to_datetime(pay_df['snapshot_createdate'])
exp_df = pay_df['snapshot_createdate'].sort_values(ascending=False).unique().tolist()

sel_date = pd.to_datetime('2022-11-30')

today_df = pay_df[pay_df['snapshot_createdate'] == sel_date]

#Testing missing today record
today_df = today_df.drop(today_df.index[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]])

yes_date = pd.to_datetime(sel_date) - pd.DateOffset(1)
yes_df = pay_df[pay_df['snapshot_createdate'] == yes_date]


#result = today_df['gidpy'].isin(yes_df['gidpy'])
#result = yes_df['gidpy'].isin(today_df['gidpy'])
result = yes_df['gidpy'].isin(today_df['gidpy'])


#print(today_df['gidpy'].isin(yes_df['gidpy']).value_counts())

